Import Library

In [325]:
import os
import numpy as np
from PIL import Image
import numpy as np
import math
from numpy.random import normal

Flatten

In [326]:
# Set the input directory path
input_directory = "../Dataset/Foto_Resize_70x70"

# List all folders in the input directory
folders = [f for f in os.listdir(input_directory) if os.path.isdir(os.path.join(input_directory, f))]
print(f"Found folders: {folders}")

# Initialize arrays for inputs and outputs
X = []
Y = []

# Create a mapping from folder names to one-hot encoded labels
label_map = {folder: idx for idx, folder in enumerate(folders)}
num_classes = len(folders)

# Process each folder and photo
for folder in folders:
    folder_path = os.path.join(input_directory, folder)
    
    # Sort the list of photos to ensure consistent order
    photos = sorted([p for p in os.listdir(folder_path) if p.endswith(('.png', '.jpg', '.jpeg'))])
    
    for photo in photos:
        photo_path = os.path.join(folder_path, photo)
        image = Image.open(photo_path)
        
        # Convert the image to a numpy array and flatten it
        image_array = np.array(image).flatten()
        X.append(image_array)
        
        # Create a one-hot encoded label
        label = np.zeros(num_classes)
        label[label_map[folder]] = 1
        Y.append(label)

# Convert lists to numpy arrays
X = np.array(X)
Y = np.array(Y)

print(f"Input array shape: {X.shape}")
print(f"Output array shape: {Y.shape}")

Found folders: ['Azmira', 'David', 'Dimas', 'Fadhli', 'Fadlin', 'Hafidz', 'Haidar', 'Hanna', 'Keiko', 'Khansa', 'Mikhael', 'Puti', 'Raesa', 'Satwika', 'Toni']


Input array shape: (1384, 4900)
Output array shape: (1384, 15)


Normalize X, Print Y

In [327]:
print(X)
xmax = X.max()
xmin = X.min()
X =  (X-xmin)/(xmax-xmin)
print(X)
np.shape(X)

[[218 220 222 ... 137 112 103]
 [253 252 251 ... 137 127 116]
 [106 117 133 ... 220 219 218]
 ...
 [213 213 214 ...  55  50  48]
 [214 214 214 ...  70  68  68]
 [ 51  48  51 ... 213 213 212]]
[[0.85490196 0.8627451  0.87058824 ... 0.5372549  0.43921569 0.40392157]
 [0.99215686 0.98823529 0.98431373 ... 0.5372549  0.49803922 0.45490196]
 [0.41568627 0.45882353 0.52156863 ... 0.8627451  0.85882353 0.85490196]
 ...
 [0.83529412 0.83529412 0.83921569 ... 0.21568627 0.19607843 0.18823529]
 [0.83921569 0.83921569 0.83921569 ... 0.2745098  0.26666667 0.26666667]
 [0.2        0.18823529 0.2        ... 0.83529412 0.83529412 0.83137255]]


(1384, 4900)

In [328]:
print(Y)
np.shape(Y)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


(1384, 15)

Initialize Layer

In [329]:
inputLayer = 4900
hiddenLayer = 1
outputLayer = 15

Calculating Weights for Input Layer to Hidden Layer

In [330]:
#70x70
#input = 4900, hiddenLayer = 1, neuron = 2

n = 15
stddev = np.sqrt(2/n)
hiddenNeurons = 15
v = np.random.normal(loc=0, scale = stddev, size = (hiddenNeurons, inputLayer))
print(v)
np.shape(v)

[[ 0.08837945  0.45740394  0.31570405 ... -0.31202378 -0.69124982
   0.10156541]
 [ 0.18810813 -0.17634221  0.09473554 ...  0.58235776 -0.35981914
   0.27236747]
 [ 0.56453585  0.88969937  0.43998735 ... -0.84671289 -0.55800248
   0.05735584]
 ...
 [ 0.14744167 -0.68509733 -0.24918747 ...  0.19574334  0.00551999
  -0.19934583]
 [-0.29132699 -0.2516339  -0.25117402 ...  0.32786125  0.36297523
  -0.27798631]
 [-0.43441826 -0.12872207  0.27631979 ...  0.06528891 -0.19888179
  -0.1851035 ]]


(15, 4900)

In [331]:
vb = np.random.normal(loc=0, scale = stddev, size = (outputLayer, 1))
print(vb)
np.shape(vb)

[[ 0.29946149]
 [-0.02740238]
 [-0.10703982]
 [ 0.50134644]
 [-0.04167503]
 [ 0.29425829]
 [-0.39916713]
 [-0.43048355]
 [ 0.35537101]
 [-0.2615296 ]
 [-0.49136259]
 [-0.9191079 ]
 [ 0.17800203]
 [ 0.04476313]
 [ 0.03318356]]


(15, 1)

Calculating Weights for Hidden Layer to Output Layer

In [332]:
#70x70
#input = 4900, hiddenLayer = 1, neuron = 2

n = 15
stddev = np.sqrt(2/n)
w = np.random.normal(loc=0, scale = stddev, size = (hiddenNeurons, outputLayer))
print(w)
np.shape(w)

[[ 0.29706906 -0.4066748  -0.16988491 -0.53257534  0.0954484  -0.03058362
  -0.28577986  0.21848269 -0.10991602  0.09472957 -0.18096585  0.30738822
   0.40011324  0.19975922 -0.07944427]
 [ 0.19860741  0.12349595  0.34728754 -0.27078535 -0.25166252  0.18167427
   0.54085614 -0.39613206 -0.36834885 -0.0872762  -0.3530097   0.53542214
  -0.4347483  -0.1883687  -0.33151278]
 [ 0.21754507  0.39526548  0.0417971  -0.41250384  0.4437786  -0.34805527
   0.77884101  0.15768381  0.05634539 -0.41365533  0.28322392 -0.46862805
   0.246671   -0.1704343   0.57879714]
 [ 0.05420139  0.56686041  0.07809348 -0.02321366 -0.26682064  0.71272745
   0.40409716  0.24426796 -0.47426172  0.34899058 -0.47634283  0.28119099
   0.21886823 -0.23611479 -0.13975418]
 [-0.41595241 -0.13521257  0.85189431  0.09790578 -0.14624368 -0.77076217
   0.1669159  -0.27728311 -0.14832678  0.20694537 -0.34324967  0.26146919
  -0.39839044 -0.56702883 -0.39126243]
 [ 0.30939702  0.39516465  0.20570727  0.34207773 -0.03249146 -0.

(15, 15)

In [333]:
print(np.shape(v))
print(np.shape(vb))
print(np.shape(w))
print(np.shape(wb))

(15, 4900)
(15, 1)
(15, 15)
(15, 1)


Calculating Bias Weight from Hidden Layer to Output Layer

In [334]:
wb = np.random.normal(loc=0, scale = stddev, size = (outputLayer, 1))
print(wb)
np.shape(wb)

[[-0.104981  ]
 [ 0.39594109]
 [ 0.31309701]
 [ 0.44504687]
 [-0.54334985]
 [ 0.00757077]
 [-0.08658901]
 [ 0.16306919]
 [-0.17090081]
 [ 0.45505455]
 [ 0.14603839]
 [-0.09389706]
 [ 0.0050927 ]
 [-0.13825954]
 [ 0.01161062]]


(15, 1)

Feedforward

In [337]:
z_in = np.dot(v,X.T) + vb
print(z_in)
np.shape(z_in)

[[  2.18426045 -20.85728954  -6.71829939 ...   4.73090028  -5.22840192
   -3.81686321]
 [ 17.64866974  12.89626206   3.01308612 ...  19.81852408   8.27436964
   20.36646647]
 [-13.94419766 -23.79989926 -10.89754335 ... -11.53947777 -23.31589961
   -6.04342736]
 ...
 [-20.66781714 -11.43242988   3.69061615 ... -15.99935652  -3.78468622
  -11.23875718]
 [-25.07843404  -9.06851095  -8.40542051 ... -29.6346952   -6.00908772
  -14.23738585]
 [ 19.55504458  -2.04801179  24.69794751 ...  25.4052325   11.91298655
   31.24768629]]


(15, 1384)

In [307]:
def ReLU(x):
    return np.maximum(0, x)

In [338]:
z = ReLU(z_in)
print(z)
np.shape(z)

[[ 2.18426045  0.          0.         ...  4.73090028  0.
   0.        ]
 [17.64866974 12.89626206  3.01308612 ... 19.81852408  8.27436964
  20.36646647]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          3.69061615 ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [19.55504458  0.         24.69794751 ... 25.4052325  11.91298655
  31.24768629]]


(15, 1384)

In [339]:
#y_in = np.dot(z,w) + wb
y_in = np.dot(w,z) + wb
print(y_in)

[[ -9.70271646  -6.28713737  -6.93346684 ... -13.49100025  -5.85204677
  -10.96655534]
 [ -2.76260973   4.31961523 -26.77322955 ...  -7.96226652  -2.52087725
  -14.26009414]
 [ 22.15198937   7.89717043  16.37378553 ...  20.44729156   7.59788816
   30.63448271]
 ...
 [ -1.23101669   6.10847826 -11.64842721 ...  -2.88195188   4.40926046
    3.41271698]
 [ 13.00047019   5.27967457   4.3620545  ...  13.96026904   6.16380176
   22.61204648]
 [-20.67014881 -10.55993213 -31.89635317 ... -23.96969662  -9.61166022
  -40.4505404 ]]


In [340]:
def Softmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / np.sum(exp_x)

In [341]:
y = Softmax(y_in)
print(y)
np.shape(y)

[[5.20674096e-21 1.58464915e-19 8.30301456e-20 ... 1.17851530e-22
  2.44844621e-19 1.47125421e-21]
 [5.37794090e-18 6.40305523e-15 2.00879830e-28 ... 2.96779415e-20
  6.84855759e-18 5.46162643e-23]
 [3.55540382e-07 2.29139377e-13 1.10013731e-09 ... 6.46469563e-08
  1.69872505e-13 1.71707389e-03]
 ...
 [2.48758789e-17 3.83072161e-14 7.43968469e-22 ... 4.77294455e-18
  7.00357323e-15 2.58539164e-15]
 [3.77081004e-11 1.67238290e-14 6.68064490e-15 ... 9.84623055e-11
  4.04862245e-14 5.63234434e-07]
 [8.98401818e-26 2.20943859e-21 1.19671852e-30 ... 3.31508916e-27
  5.70310272e-21 2.30651193e-34]]


(15, 1384)

Back Propagation

In [342]:
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size,Y.max()+1))
    one_hot_Y [np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

In [345]:
sigma_k = y-Y.T
print(sigma_k)
np.shape(sigma_k)

[[-1.00000000e+00 -1.00000000e+00 -1.00000000e+00 ...  1.17851530e-22
   2.44844621e-19  1.47125421e-21]
 [ 5.37794090e-18  6.40305523e-15  2.00879830e-28 ...  2.96779415e-20
   6.84855759e-18  5.46162643e-23]
 [ 3.55540382e-07  2.29139377e-13  1.10013731e-09 ...  6.46469563e-08
   1.69872505e-13  1.71707389e-03]
 ...
 [ 2.48758789e-17  3.83072161e-14  7.43968469e-22 ...  4.77294455e-18
   7.00357323e-15  2.58539164e-15]
 [ 3.77081004e-11  1.67238290e-14  6.68064490e-15 ...  9.84623055e-11
   4.04862245e-14  5.63234434e-07]
 [ 8.98401818e-26  2.20943859e-21  1.19671852e-30 ... -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00]]


(15, 1384)

In [347]:
delta_w = (1/hiddenNeurons)*np.dot(sigma_k, z.T)
print(delta_w)



[[-6.76401605e+00 -4.99380002e+01  5.12833653e-18 -1.74176115e+01
  -3.31118301e+00 -4.59992194e+00 -3.22310550e+01 -3.23572321e+01
  -5.20685519e+01  9.08606379e-22  7.82755128e-20 -6.28209147e+00
  -2.24560581e+00  4.09750201e-21 -5.77856610e+01]
 [-3.07724331e-01 -1.39515681e+02  7.22534692e-17 -1.77598752e+01
   4.57561300e-13 -8.82617110e+00 -4.59273715e+01 -3.28673074e+01
  -4.60676223e+01  6.50791792e-20  1.93406573e-18 -1.42097947e-01
   1.70219928e-18  1.44835957e-17 -1.46305684e+02]
 [-3.24136254e+00 -6.22210874e+01  3.15425109e-03 -2.27126815e+01
  -1.12569721e+00 -1.60114606e+00 -4.59246563e+01 -2.54017688e+01
  -3.74294159e+01  3.34188225e-18  3.38101583e-12 -4.16997330e+00
  -5.37453517e-02  1.60038303e-06 -5.34246844e+01]
 [ 3.64654111e-08 -9.93017453e+01  1.65351448e-11 -1.01399650e+01
   3.43969772e-08 -5.94708301e-01 -4.81648613e+01 -1.18054017e+01
  -3.45784774e+01  6.22491507e-09  1.43394198e-08  1.63542694e-08
   2.29727004e-14  9.15963009e-09 -1.04136762e+02]
 [-4

In [350]:
delta_wb = (1/hiddenNeurons)*np.sum(sigma_k)
print(delta_wb)

-92.2


In [352]:
def d_ReLU(x):
    return x>0

In [356]:
sigma_in = np.dot(wb.T, sigma_k)*d_ReLU(z_in)
print(sigma_in)
np.shape(sigma_in)

[[ 0.10498111  0.          0.         ... -0.0116106  -0.
  -0.        ]
 [ 0.10498111  0.104981    0.104981   ... -0.0116106  -0.01161062
  -0.01107309]
 [ 0.          0.          0.         ... -0.         -0.
  -0.        ]
 ...
 [ 0.          0.          0.104981   ... -0.         -0.
  -0.        ]
 [ 0.          0.          0.         ... -0.         -0.
  -0.        ]
 [ 0.10498111  0.          0.104981   ... -0.0116106  -0.01161062
  -0.01107309]]


(15, 1384)

In [363]:
delta_v = (1/hiddenNeurons)*np.dot(sigma_in, X)
print(delta_v)
np.shape(delta_v)

[[ 0.09432992  0.09297253  0.10891153 ... -0.08658176 -0.03070763
   0.02909363]
 [-2.79235368 -2.90591851 -2.96460664 ... -3.00883332 -2.9264201
  -2.79790072]
 [-0.00433201 -0.00468256 -0.00420282 ... -0.01188693 -0.01219784
  -0.01247025]
 ...
 [-0.01949741 -0.01339035 -0.00958127 ...  0.02234829  0.01923551
   0.02243454]
 [-0.02716691 -0.02268585 -0.01727482 ... -0.02769253 -0.02721278
  -0.02723131]
 [-2.82520047 -2.93630084 -2.99165773 ... -3.01891379 -2.9269194
  -2.79089268]]


(15, 4900)

In [365]:
delta_vb = (1/hiddenNeurons)*np.sum(sigma_in)
print(delta_vb)

-18.204625299081147


Update Parameters

In [368]:
alpha = 0.2
w = w - alpha*delta_w
print(w)
wb = wb - alpha*delta_wb
print(wb)
v = v - alpha*delta_v
print(v)
vb = vb- alpha*delta_vb
print(vb)

[[ 3.00267549e+00  1.95685253e+01 -1.69884915e-01  6.43446925e+00
   1.41992161e+00  1.80938515e+00  1.26066421e+01  1.31613755e+01
   2.07175047e+01  9.47295698e-02 -1.80965847e-01  2.82022481e+00
   1.29835556e+00  1.99759223e-01  2.30348201e+01]
 [ 3.21697140e-01  5.59297685e+01  3.47287535e-01  6.83316472e+00
  -2.51662522e-01  3.71214271e+00  1.89118047e+01  1.27507909e+01
   1.80587001e+01 -8.72761965e-02 -3.53009699e-01  5.92261322e-01
  -4.34748303e-01 -1.88368696e-01  5.81907607e+01]
 [ 1.51409009e+00  2.52837004e+01  4.05353996e-02  8.67256876e+00
   8.94057485e-01  2.92403151e-01  1.91487035e+01  1.03183913e+01
   1.50281117e+01 -4.13655330e-01  2.83223922e-01  1.19936127e+00
   2.68169142e-01 -1.70434939e-01  2.19486709e+01]
 [ 5.42013728e-02  4.02875585e+01  7.80934768e-02  4.03277233e+00
  -2.66820655e-01  9.50610773e-01  1.96700417e+01  4.96642865e+00
   1.33571292e+01  3.48990582e-01 -4.76342835e-01  2.81190987e-01
   2.18868230e-01 -2.36114792e-01  4.15149507e+01]
 [ 1